<a href="https://colab.research.google.com/github/sudharsan99-hacker/PersonalGPT/blob/main/PersonalGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyMuPDF
import fitz  # PyMuPDF
import re
import os

In [2]:
!pip install langchain-community
!pip install unstructured
!pip install nltk
from langchain_community.document_loaders import DirectoryLoader

In [3]:
!pip install sentence-transformers==2.2.2
!pip install langchain-community


In [4]:
!pip uninstall nltk -y
!pip install nltk
import nltk
nltk.download('punkt')

nltk.data.path.append('/root/nltk_data')
from langchain_community.document_loaders import DirectoryLoader

directory_path = './personalDocuments'
loader = DirectoryLoader(directory_path, glob="**/*")
docs = loader.load()


Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)
text_chunks = text_splitter.split_documents(docs)


In [6]:
!pip install faiss-gpu
!pip install faiss-cpu
!pip install InstructorEmbedding
from sentence_transformers import SentenceTransformer
from InstructorEmbedding import INSTRUCTOR
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

# Initialize the embedding model
embedding_model = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-large')

# Create the FAISS vector store
vector_store = FAISS.from_documents(text_chunks, embedding_model, normalize_L2=True)

# Save the vector store locally
vector_store.save_local('vectorstore')


load INSTRUCTOR_Transformer


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no lon

max_seq_length  512


In [66]:
!pip install --upgrade streamlit
from langchain_community.vectorstores import FAISS
from InstructorEmbedding import INSTRUCTOR
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

import streamlit as st
import os
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from dotenv import load_dotenv

In [67]:
# Set Streamlit page configuration with title and layout
st.set_page_config(page_title='Sudharsan\'s Personal AI Assistant 🤖', layout='wide')

2024-10-12 20:19:19.170 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [68]:

import streamlit as st
# Load the model if not already loaded in cache_resource
@st.cache_resource()
def load_model():
    return HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-xl')

embedding_model = load_model()

# Load the vectordb from local storage
new_db = FAISS.load_local("vectorstore", embedding_model, allow_dangerous_deserialization=True)


2024-10-12 20:19:21.548 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 20:19:21.550 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 20:19:21.554 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 20:19:21.570 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 20:19:21.572 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [69]:
# Retriever
retriever = new_db.as_retriever(search_kwargs={"k": 7, "hnsw:space": "cosine"})

In [70]:
!pip install together
!pip install langchain-together
from together import Together
from langchain_together.chat_models import ChatTogether
from dotenv import load_dotenv

# Set api key
load_dotenv("D:\FarmwiseAI\Reddit\.env")

# Load the API key from the environment
api_key = '6b0637401d057af5d54b59bdfa0cb55d1b4c35115b3403d27cbed6720b453744'
os.environ["TOGETHER_API_KEY"] = api_key
llm = ChatTogether(model="meta-llama/Llama-3-70b-chat-hf", temperature=0, max_tokens=2000,model_kwargs={"top_p": 0.5})

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'top_p'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


In [71]:

# Initialize the language model with the API key
llm = ChatTogether(
    openai_api_key=api_key,
    model="meta-llama/Llama-3-70b-chat-hf",
    temperature=0,
    max_tokens=2000,
    model_kwargs={"top_p": 0.5}
)


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'top_p'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


In [72]:
# Prompt for condensing the chat history
condense_q_system_prompt = """Given a chat history and the latest user question \
which might reference the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
condense_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
# Chain for condensing the question
condense_q_chain = condense_q_prompt | llm | StrOutputParser()

In [73]:
# System Prompt for the AI Assistant
qa_system_prompt = """You are an AI assistant designed to provide comprehensive information about Sudharsan. Your primary role is to help users understand various aspects of Sudharsan's life, work, interests, achievements, etc. based on the context provided and the question asked by them. Below is a detailed guide on how to interact and provide valuable assistance:

User Profile:
Name: Sudharsan
Profession: SQL Developer
Experience: 10 months working with SQL databases and ETL workflows to build applications such as a relational database for insurance claim scenarios and an automated data retrieval system.
Interests: Tech, Algorithm design and optimization, health, fitness, longevity, self-help podcasts, and working out.

Interaction Guidelines:
* Be Clear and Informative: Provide detailed and accurate information about Sudharsan. Avoid overly technical jargon unless necessary.
* Be Supportive and Encouraging: Encourage users to learn more about Sudharsan's interests and achievements. Offer positive reinforcement.
* Be Detailed and Comprehensive: Provide in-depth information about Sudharsan's work, interests, and achievements. Include relevant examples and details.
* Make sure to only provide information about Sudharsan from the parts of the context that is relevant to the question asked.

Example Dialogue:
User: Can you tell me about Sudharsan's professional background?

AI Assistant: Sudharsan is a SQL Developer with 9 months of experience designing and optimizing relational databases for complex insurance claim scenarios. He has implemented ETL workflows using SQL Server Integration Services (SSIS), streamlining data extraction, transformation, and loading processes. Additionally, he has developed and refined SQL queries and stored procedures, focusing on optimization to enhance efficiency and automate routine database tasks. Sudharsan is passionate about leveraging innovative technologies to drive results in his work.

Context:
{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def format_docs(documents):
    return "\n\n".join(document.page_content for document in documents)

def condense_question(input: dict):
    if input.get("chat_history"):
        return condense_q_chain
    else:
        return input["question"]

rag_chain = (
    RunnablePassthrough.assign(context=condense_question | retriever | format_docs)
    | qa_prompt
    | llm
)

In [74]:
# Sample question
rag_chain.invoke({"question": "What is the professional background of Sudharsan?", "chat_history": []})

AIMessage(content='Sudharsan is a SQL Developer with 10 months of experience working with SQL databases and ETL workflows to build applications such as a relational database for insurance claim scenarios and an automated data retrieval system. He has implemented ETL workflows using SQL Server Integration Services (SSIS), streamlining data extraction, transformation, and loading processes. Additionally, he has developed and refined SQL queries and stored procedures, focusing on optimization to enhance efficiency and automate routine database tasks. Sudharsan is passionate about leveraging innovative technologies to drive results in his work.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 1029, 'total_tokens': 1140, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta-llama/Llama-3-70b-chat-hf', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-699471c8-59

In [75]:
# Initialize chat history
chat_history = []

# Streamlit app setup
st.title("Sudharsan's Personal AI Assistant")

# Function to handle message sending
def send_message():
    user_input = st.session_state.user_input
    if user_input:
        # Process user message
        ai_msg = rag_chain.invoke({"question": user_input, "chat_history": st.session_state['chat_history']})
        st.session_state['chat_history'].extend([HumanMessage(content=user_input), ai_msg])
        # Print chat history for debugging
        print("Updated chat history:", st.session_state['chat_history'])


        # Clear the input box
        st.session_state.user_input = ""

# Use columns for better layout - col 1 for chat history and col 2 for user input
col1, col2 = st.columns([3, 1])

# Container for chat history
# Initialize session state for chat history if not already present
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []

# Container for chat history
with col1:
    with st.container():
        for message in st.session_state['chat_history']:
            if isinstance(message, HumanMessage):
                st.markdown(f"""**USER** 🧑‍💻:
                            {message.content}""")
                st.write('------------')
            else:
                st.markdown(f"""**AI** 🤖:
                            {message.content}""")
                st.write('------------')

# Container for user input
with col2:
    with st.container():
        # Text input for user message
        user_input = st.text_area("Your message:", key="user_input", args=(), height=400)

        # Send button
        send_btn = st.button("Send", on_click=send_message)

# CSS for making the first column scrollable
css = '''
<style>
    /*section.main > div {
        /* padding-bottom: 1rem; */
    }
    [data-testid="column"] > div > div > div > div > div {
        overflow: auto;
        height: 80vh;
    }
    .element-container {
        margin-bottom: 0rem !important;
    }
    .stTextArea {
        margin-bottom: 1rem !important;
    }*/
</style>
'''

st.markdown(css, unsafe_allow_html=True)

# JavaScript to manage the scroll position
scroll_js = """
<script>
function maintainScrollPosition(){
    var container = document.querySelector("[data-testid='column']>div>div>div>div>div");
    var scrollPosition = localStorage.getItem('scrollPosition');
    if (scrollPosition) {
        container.scrollTop = scrollPosition;
    }
    container.addEventListener('scroll', function() {
        localStorage.setItem('scrollPosition', container.scrollTop);
    });
}
// Call the function when the page loads
setTimeout(maintainScrollPosition, 400);
</script>
"""

# Render the JavaScript to manage the scroll position
st.markdown(scroll_js, unsafe_allow_html=True)

2024-10-12 20:20:06.782 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 20:20:06.784 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 20:20:06.787 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 20:20:06.789 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 20:20:06.791 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 20:20:06.797 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 20:20:06.799 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 20:20:06.801 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [77]:
!pip install pyngrok
import os
from pyngrok import ngrok
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceInstructEmbeddings


# Use your actual authtoken here
authtoken = '2nLdmJKvU5Clw4mZo8ZkJoXWtfc_4owKkQBmS7mVSHvCkokby'
# Set the authtoken using the pyngrok library
ngrok.set_auth_token(authtoken)

# Configure ngrok tunnel correctly
public_url = ngrok.connect(8501)
print(f"Streamlit app will be available on: {public_url}")
# Continue with Streamlit setup
os.system('streamlit run streamlit_app.py &> /dev/null &')


Streamlit app will be available on: NgrokTunnel: "https://d05d-35-229-219-59.ngrok-free.app" -> "http://localhost:8501"


0